In [2]:
import torch
from torch import nn
from torch.optim import Adam
import torchaudio
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from skimage.util import img_as_ubyte
import pandas as pd
import os
import glob
import numpy as np
import librosa
import matplotlib.pyplot as plt
import sys
import io
from sklearn.metrics import roc_auc_score

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
class MimiiDataset(Dataset):
    def __init__(self,audio_dir, n_fft = 1024, win_length = 1024,
                 hop_length = 512,power = 2,n_mels = 128,pad_mode = 'reflect',
                 sr = 16000,center = True,norm = None):
      
        super(MimiiDataset, self).__init__()
        self.audio_dir = audio_dir
        self.n_mels = n_mels
        self.n_fft = n_fft
        self.win_length = win_length
        self.hop_length = hop_length
        self.power = power
        self.pad_mode = pad_mode
        self.sr = sr
        self.center = center
        self.norm = norm

    def get_files(self):
       return self.train_files, self.test_files
    
    def get_data(self,device, id):
        
        self.train_files, self.train_labels = self._train_file_list(device, id)
        self.test_files, self.test_labels = self._test_file_list(device, id)
        
        self.train_data = self.get_audios(self.train_files)
        self.test_data = self.get_audios(self.test_files)
        
        return self.train_data, self.test_data, self.train_labels, self.test_labels
    
    def _train_file_list(self, device, id):
        query = os.path.abspath(
            f"{self.audio_dir}/{device}/train/normal_id_0{id}*.wav"
        )
        train_normal_files = sorted(glob.glob(query))
        train_normal_labels = np.zeros(len(train_normal_files))
        
        query = os.path.abspath(
                f"{self.audio_dir}/{device}/train/anomaly_id_0{id}*.wav"
            )
        train_anomaly_files = sorted(glob.glob(query))
        train_anomaly_labels = np.ones(len(train_anomaly_files))
        
        train_file_list = np.concatenate((train_normal_files, train_anomaly_files), axis=0)
        train_labels = np.concatenate((train_normal_labels, train_anomaly_labels), axis=0)
        
        return train_file_list, train_labels
    
    def _test_file_list(self, device, id):     
        query = os.path.abspath(
            f"{self.audio_dir}/{device}/test/normal_id_0{id}*.wav"
            )
        test_normal_files = sorted(glob.glob(query))
        test_normal_labels = np.zeros(len(test_normal_files))
        
        query = os.path.abspath(
            f"{self.audio_dir}/{device}/test/anomaly_id_0{id}*.wav"
            )
        test_anomaly_files = sorted(glob.glob(query))
        test_anomaly_labels = np.ones(len(test_anomaly_files))
        
        test_file_list = np.concatenate((test_normal_files, 
                                          test_anomaly_files), axis=0)
        test_labels = np.concatenate((test_normal_labels,
                                      test_anomaly_labels), axis=0)
          
        return test_file_list, test_labels

    def normalize(self,tensor):
        tensor_minusmean = tensor - tensor.mean()
        return tensor_minusmean/np.absolute(tensor_minusmean).max()

    def make0min(self,tensornd):
        tensor = tensornd.numpy()
        res = np.where(tensor == 0, 1E-19 , tensor)
        return torch.from_numpy(res)

    def spectrogrameToImage(self,specgram):
        # specgram = torchaudio.transforms.MelSpectrogram(n_fft=1024, win_length=1024, 
        #                                                 hop_length=512, power=2, 
        #                                                 normalized=True, n_mels=128)(waveform )
        specgram= self.make0min(specgram)
        specgram = specgram.log2()[0,:,:].numpy()
        
        tr2image = transforms.Compose([transforms.ToPILImage()])

        specgram= self.normalize(specgram)
        # specgram = img_as_ubyte(specgram)
        specgramImage = tr2image(specgram)
        return specgramImage

    def get_logmelspectrogram(self, waveform):
        melspec = librosa.feature.melspectrogram(
          n_fft=self.n_fft, win_length=self.win_length, hop_length=self.hop_length,
          power=self.power,n_mels=self.n_mels,pad_mode=self.pad_mode,sr=self.sr,
          center=self.center,norm=self.norm,htk=True,
          y = waveform.numpy()
        )

        logmelspec = librosa.power_to_db(melspec)

        return logmelspec

    def get_melspectrogram(self,waveform):
        melspec = librosa.feature.melspectrogram(
            n_fft=self.n_fft, win_length=self.win_length, hop_length=self.hop_length,
            power=self.power,n_mels=self.n_mels,pad_mode=self.pad_mode,sr=self.sr,
            center=self.center,norm=self.norm,htk=True,
            y = waveform.numpy()
        )

        return melspec
    
    def get_mfcc(self,waveform):
        mfcc = librosa.feature.mfcc(    
            n_fft=self.n_fft, win_length=self.win_length, 
            hop_length=self.hop_length,pad_mode=self.pad_mode,sr=self.sr,
            center=self.center,norm=self.norm,n_mfcc=40,
            y = waveform.numpy()
        )

        return mfcc

    def get_chroma_stft(self,waveform):
        stft = librosa.feature.chroma_stft(
            n_fft=self.n_fft, win_length=self.win_length, 
            hop_length=self.hop_length,pad_mode=self.pad_mode,sr=self.sr,
            center=self.center,norm=self.norm,n_chroma=12,
            y=waveform.numpy()
        )

        return stft

    def get_spectral_contrast(self,waveform):
        spec_contrast = librosa.feature.spectral_contrast(    
            n_fft=self.n_fft, win_length=self.win_length,center=self.center,
            hop_length=self.hop_length,pad_mode=self.pad_mode,sr=self.sr,
            y = waveform.numpy()
        )

        return spec_contrast
    
    def get_tonnetz(self,waveform):
        harmonic = librosa.effects.harmonic(waveform.numpy())
        tonnetz = librosa.feature.tonnetz(y=harmonic,sr=self.sr)

        return tonnetz

    def get_audios(self, file_list):
        data = []
        for i in range(len(file_list)):
          y, sr = torchaudio.load(file_list[i])  
          data.append(y)

        return data
    def _derive_data(self, file_list):
        train_data = []
        test_data = []
        train_mode = True
        for file_list in [self.train_files, self.test_files]:
          tr2tensor = transforms.Compose([transforms.PILToTensor()])
          data = []
          for j in range(len(file_list)):
            y, sr = torchaudio.load(file_list[j])  
            spec = self.get_melspectrogram(y)
            spec = self.spectrogrameToImage(spec)
            spec = spec.convert('RGB')
            vectors = tr2tensor(spec)
            if train_mode:     
              train_data.append(vectors)
            else:
              test_data.append(vectors)
            
          train_mode = False
                
        return data

In [6]:
dataset = MimiiDataset('/content/drive/MyDrive/mimii')

In [13]:
_, _, y_train_fan0, y_test_fan0 = dataset.get_data('slider', 0)
_, _, y_train_fan2, y_test_fan2 = dataset.get_data('slider', 2)
_, _, y_train_fan4, y_test_fan4 = dataset.get_data('slider', 4)
_, _, y_train_fan6, y_test_fan6 = dataset.get_data('slider', 6)

In [146]:
def mean_mfccs(wave_list):
  data = []
  for wave in wave_list:
    mfcc = np.mean(dataset.get_mfcc(wave)[0], axis = 1)
    data.append(mfcc)
  
  return data

def mean_stfts(wave_list):
  data = []
  for wave in wave_list:
    stft = np.mean(dataset.get_chroma_stft(wave)[0], axis = 1)
    data.append(stft)
  
  return data

def mean_melspecs(wave_list):
  data = []
  for wave in wave_list:
    melspec = np.mean(dataset.get_melspectrogram(wave)[0], axis = 1)
    data.append(melspec)
  
  return data

def mean_spec_contrasts(wave_list):
  data = []
  for wave in wave_list:
    spec_contrast = np.mean(dataset.get_spectral_contrast(wave)[0], axis = 1)
    data.append(spec_contrast)
  
  return data
  
def mean_tonnetzs(wave_list):
  data = []
  for wave in wave_list:
    tonnetz = np.mean(dataset.get_tonnetz(wave)[0], axis = 1)
    data.append(tonnetz)
  
  return data

In [7]:
# df_train1, df_test1, y_train1, y_test1 = dataset.get_data('ToyConveyor', 1)
# df_train2, df_test2, y_train2, y_test2 = dataset.get_data('ToyConveyor', 2)
# df_train3, df_test3, y_train3, y_test3 = dataset.get_data('ToyConveyor', 3)
df_train4, df_test4, y_train4, y_test4 = dataset.get_data('ToyConveyor', 4)

In [11]:
len(df_test4)

0

In [12]:
train_melspecs, test_melspecs = mean_melspecs(df_train1), mean_melspecs(df_test1)
train_mfccs, test_mfccs = mean_mfccs(df_train1), mean_mfccs(df_test1)
train_stfts, test_stfts = mean_stfts(df_train1), mean_stfts(df_test1)
train_spec_contrasts, test_spec_contrasts = mean_spec_contrasts(df_train1), mean_spec_contrasts(df_test1)
train_tonnetzs, test_tonnetzs = mean_tonnetzs(df_train1), mean_tonnetzs(df_test1)

train_mixed_f = []
test_mixed_f = []

for i in range(len(train_melspecs)):

  train_mf = np.concatenate((train_melspecs[i],train_mfccs[i],train_stfts[i],
                             train_spec_contrasts[i],train_tonnetzs[i])).tolist()

  train_mixed_f.append(train_mf)

for i in range(len(test_melspecs)):

  test_mf = np.concatenate((test_melspecs[i],test_mfccs[i],test_stfts[i],
                             test_spec_contrasts[i],test_tonnetzs[i])).tolist()

  test_mixed_f.append(test_mf)

train_mixed_f = torch.FloatTensor(train_mixed_f)
test_mixed_f = torch.FloatTensor(test_mixed_f)

In [149]:
torch.save(train_mixed_f, '/content/drive/MyDrive/mixed_features/y_tr_toyconveyor1.pt')
torch.save(test_mixed_f, '/content/drive/MyDrive/mixed_features/y_ts_toyconveyor1.pt')

In [14]:
torch.save(torch.FloatTensor(y_train_fan0.tolist()), '/content/drive/MyDrive/labels/y_tr_slider0.pt')
torch.save(torch.FloatTensor(y_test_fan0.tolist()), '/content/drive/MyDrive/labels/y_ts_slider0.pt')
torch.save(torch.FloatTensor(y_train_fan2.tolist()), '/content/drive/MyDrive/labels/y_tr_slider2.pt')
torch.save(torch.FloatTensor(y_test_fan2.tolist()), '/content/drive/MyDrive/labels/y_ts_slider2.pt')
torch.save(torch.FloatTensor(y_train_fan4.tolist()), '/content/drive/MyDrive/labels/y_tr_slider4.pt')
torch.save(torch.FloatTensor(y_test_fan4.tolist()), '/content/drive/MyDrive/labels/y_ts_slider4.pt')
torch.save(torch.FloatTensor(y_train_fan6.tolist()), '/content/drive/MyDrive/labels/y_tr_slider6.pt')
torch.save(torch.FloatTensor(y_test_fan6.tolist()), '/content/drive/MyDrive/labels/y_ts_slider6.pt')

In [15]:
y_train, y_test = torch.load('/content/drive/MyDrive/labels/y_tr_slider0.pt'), torch.load('/content/drive/MyDrive/labels/y_ts_slider0.pt')

In [30]:
train_mf, test_mf = torch.load('/content/drive/MyDrive/mixed_features/train_mf_slider0.pt'), torch.load('/content/drive/MyDrive/mixed_features/test_mf_slider0.pt')

In [31]:
train_data = DataLoader(train_mf, batch_size=64, shuffle = True)
test_data = DataLoader(test_mf, batch_size = 64, shuffle= False)

In [32]:
class UNet_FC(nn.Module):

  def __init__(self, in_features):
    super().__init__()
    
    self.bn = nn.BatchNorm1d(128)
    self.relu = nn.ReLU()

    self.fc0 = nn.Linear(in_features=in_features,out_features=in_features)

    # encoder
    self.fc1 = nn.Linear(in_features=in_features, out_features=128)
    self.fc2 = nn.Linear(in_features=128,out_features=128)
    self.fc3 = nn.Linear(in_features=128, out_features=128)
    self.fc4 = nn.Linear(in_features=128, out_features=128)
    self.fc5 = nn.Linear(in_features=128, out_features=8)

    # decoder
    self.fc6 = nn.Linear(in_features=8, out_features=128)
    self.fc7 = nn.Linear(in_features=128*2, out_features=128)
    self.fc8 = nn.Linear(in_features=128*2, out_features=128)
    self.fc9 = nn.Linear(in_features=128*2, out_features=128)

    self.out = nn.Linear(in_features=128*2, out_features=in_features)

  def encoder(self, x):
    input = self.fc0(x)

    x1 = self.relu(self.bn(self.fc1(input)))
    x2 = self.relu(self.bn(self.fc2(x1)))
    x3 = self.relu(self.bn(self.fc3(x2)))
    x4 = self.relu(self.bn(self.fc4(x3)))
    x5 = self.relu(self.fc5(x4))

    return [x5, x4, x3, x2, x1]

  def decoder(self, x):
    x6 = self.relu(self.fc6(x[0]))
    con1 = torch.cat((x6,x[1]), 1) 
    x7 = self.relu(self.bn(self.fc7(con1)))
    con2 = torch.cat((x7,x[2]), 1)
    x8 = self.relu(self.bn(self.fc8(con2)))
    con3 = torch.cat((x8,x[3]), 1)
    x9 = self.relu(self.bn(self.fc9(con3)))
    con4 = torch.cat((x9,x[4]), 1)

    x10 = self.out(con4)

    return x10
  
  def forward(self, x):
    # encoded = self.encoder(x)

    # decoded = self.decoder(encoded)
    input = self.fc0(x)

    x1 = self.relu(self.bn(self.fc1(input)))
    x2 = self.relu(self.bn(self.fc2(x1)))
    x3 = self.relu(self.bn(self.fc3(x2)))
    x4 = self.relu(self.bn(self.fc4(x3)))
    x5 = self.relu(self.fc5(x4))

    xy = [x5, x4, x3, x2, x1]

    x6 = self.relu(self.fc6(xy[0]))
    con1 = torch.cat((x6,xy[1]), 1) 
    x7 = self.relu(self.bn(self.fc7(con1)))
    con2 = torch.cat((x7,xy[2]), 1)
    x8 = self.relu(self.bn(self.fc8(con2)))
    con3 = torch.cat((x8,xy[3]), 1)
    x9 = self.relu(self.bn(self.fc9(con3)))
    con4 = torch.cat((x9,xy[4]), 1)

    x10 = self.out(con4)

    # return decoded
    return x10

In [33]:
def train(model, optimizer, criterion, data_tr, data_val, scheduler = None,
          epochs = 3000, device = 'cpu'):
    # X_val, Y_val = next(iter(data_val))
    losses = []
    prev_avg_loss = 100000
    for epoch in range(epochs):
        train_avg_loss = 0
        test_avg_loss = 0
        # model.train()  # train mode
        for batch in data_tr:
          # data to device
          batch = batch.to(device)
          # set parameter gradients to zero
          optimizer.zero_grad()
          # forward
          # print(Y_batch.shape)
          predictions = model(batch)
          loss = criterion(predictions, batch)
          loss.backward() # backward-pass
          optimizer.step()  # update weights
          # calculate loss to show the user
          if scheduler:
            scheduler.step(loss)
          train_avg_loss += loss / len(data_tr)

        # model.eval()
        for batch in data_val:
          with torch.no_grad():
            preds = model(batch.to(device)).cpu()
            loss = criterion(preds,batch)
            test_avg_loss += loss / len(data_val)
                    
        losses.append(train_avg_loss.item())
        if (epoch+1)%100 == 0:
          print("{}/{} train_loss: {} test_loss:{}".format(epoch+1, epochs, train_avg_loss, test_avg_loss))
        # if test_avg_loss < 70:
        #   break
    return losses

In [34]:
unet = UNet_FC(in_features=193).to(device)
optimizer = Adam(params = unet.parameters(), lr = 10e-3)
# optimizer = Adam(params = unet.parameters())
criterion = nn.MSELoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, 
                                                       min_lr=10e-4, mode = 'min',
                                                       patience = 30)

In [35]:
losses= train(model = unet, optimizer = optimizer, criterion=criterion, data_tr=train_data,
               data_val = test_data, scheduler = scheduler, device = device)

100/3000 train_loss: 548.6768188476562 test_loss:3930.87841796875
200/3000 train_loss: 373.0775451660156 test_loss:3878.34326171875
300/3000 train_loss: 250.9510498046875 test_loss:3912.1787109375
400/3000 train_loss: 226.56639099121094 test_loss:3917.408203125
500/3000 train_loss: 176.40565490722656 test_loss:3743.869873046875
600/3000 train_loss: 144.66822814941406 test_loss:3580.272216796875
700/3000 train_loss: 126.55216979980469 test_loss:3696.25244140625
800/3000 train_loss: 113.44833374023438 test_loss:3564.515380859375
900/3000 train_loss: 98.3164291381836 test_loss:3522.873291015625
1000/3000 train_loss: 93.22779846191406 test_loss:3508.326904296875
1100/3000 train_loss: 86.74024963378906 test_loss:3435.44775390625
1200/3000 train_loss: 71.61650848388672 test_loss:3583.8994140625
1300/3000 train_loss: 71.25789642333984 test_loss:3567.39306640625
1400/3000 train_loss: 63.721744537353516 test_loss:3679.42431640625
1500/3000 train_loss: 58.28046417236328 test_loss:3507.9091796875

In [36]:
avg_loss = 0
# unet.eval()
# unet.train()
preds = []
i = 0
test_anomaly_losses = []
test_normal_losses = []
test_losses = []
# test_real = y_test_fan0.tolist()
# y_test_fan0 = y_test_fan0.tolist()
for batch in test_data:
  with torch.no_grad():
    # unet.train()
    predictions = unet(batch.to(device)).cpu()
    preds.append(predictions)
  loss = criterion(predictions, batch.cpu())
  for j in range(len(predictions)):
    if int(y_test[i]) == 1:
      test_anomaly_losses.append(float(criterion(predictions[j], batch[j])))
    else:
      test_normal_losses.append(float(criterion(predictions[j], batch[j])))
    i += 1
    test_losses.append(criterion(predictions[j], batch[j]))
  # print(loss)
  # print(loss)
  avg_loss += loss / len(test_data)
# avg_loss

avg_loss

tensor(3537.0271)

In [37]:
sum(test_anomaly_losses)/len(test_anomaly_losses) , sum(test_normal_losses)/len(test_normal_losses)

(4364.920546542393, 217.59184698104858)

In [38]:
# torch.save(unet, "unet_fan2_2")

In [39]:
len(preds)

8

In [41]:
results = []
vals = np.arange(40, 900, 0.5).tolist()
for threshold in vals:
  preds = []
  for j in range(len(test_losses)):
    if test_losses[j] > threshold:
      preds.append(1)
    else:
      preds.append(0)
  
  results.append(roc_auc_score(y_test,preds))

print(max(results))

0.9705056179775281


In [ ]:
def get_logmelspectrogram(waveform):
    melspec = librosa.feature.melspectrogram(y=waveform.numpy(), hop_length=250, n_mels = 304)

    logmelspec = librosa.power_to_db(melspec)

    return logmelspec

In [ ]:
# train_logmelspecs, test_logmelspecs = mean_logmelspecs(df_train), mean_logmelspecs(df_test)
train_data1 = []
for wave in df_train:
  train_data1.append(get_logmelspectrogram(wave)[0])

test_data1 = []
for wave in df_test:
  test_data1.append(get_logmelspectrogram(wave)[0])

In [ ]:
train_data1 = torch.FloatTensor(train_data1)
test_data1 = torch.FloatTensor(test_data1)

<ipython-input-30-68ec04120629>:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  train_data1 = torch.FloatTensor(train_data1)


In [ ]:
train_logs = DataLoader(train_data1.reshape(916*304,641),batch_size = 304)
test_logs = DataLoader(test_data1.reshape(459*304,641),batch_size = 304)

In [ ]:
unet1 = UNet_FC(in_features=641).to(device)
optimizer1 = Adam(params = unet1.parameters(), lr = 10e-3)
# optimizer = Adam(params = unet.parameters())
criterion1 = nn.MSELoss()
scheduler1 = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer1, factor=0.5, 
                                                       min_lr=10e-4, mode = 'min',
                                                       patience = 30)

In [ ]:
losses = train(model = unet1, optimizer = optimizer1, criterion=criterion1, data_tr=train_logs,
               data_val = test_logs, device = device, epochs = 1)

1/1 train_loss: 21.703369140625 test_loss:17.446470260620117


In [ ]:
avg_loss = 0
# unet.eval()
# unet.train()
preds = []
i = 0
test_anomaly_losses = []
test_normal_losses = []
test_losses = []
test_real = y_test.tolist()
for batch in test_logs:
  with torch.no_grad():
    # unet.train()
    predictions = unet1(batch.to(device)).cpu()
    preds.append(predictions)
    loss = criterion(predictions, batch.cpu())
    test_losses.append(loss)
    if int(y_test[i]) == 1:
      test_anomaly_losses.append(loss)
    else:
      test_normal_losses.append(loss)
    i += 1
  # print(loss)
  # print(loss)
  avg_loss += loss / len(test_logs)
# avg_loss

avg_loss

tensor(17.4465)

In [ ]:
sum(test_anomaly_losses)/len(test_anomaly_losses) , sum(test_normal_losses)/len(test_normal_losses)

(tensor(18.0466), tensor(15.2920))

In [ ]:
results = []
vals = np.arange(10, 21, 0.1).tolist()
for threshold in vals:
  preds = []
  for j in range(len(test_losses)):
    if test_losses[j] > threshold:
      preds.append(1)
    else:
      preds.append(0)
  
  results.append(roc_auc_score(test_real,preds))

print(max(results))

0.5875626740947075
